# Acceso a Bases de Datos

Para conectarnos desde nuestro código Java a una base de datos necesitamos un driver que nos permita realizar la conexión. En este caso vamos a utilizar el driver de MySQL.

El driver de MySQL se puede descargar desde la página oficial de MySQL: [https://dev.mysql.com/downloads/connector/j/](https://dev.mysql.com/downloads/connector/j/). Nos descargaremos la versión "Platform Independent" en formato ZIP.

Una vez descargado el driver, lo descomprimimos y escogemos el fichero `mysql-connector-java-8.0.23.jar` haciendo botón derecho sobre las librerías de nuestro proyecto y seleccionando "Add JAR/Folder".

El proceso para conectarnos a una base de datos MySQL desde Java es el siguiente:

- Establecer los datos de conexión y conectarnos a la base de datos mediante un objeto `Connection`.
- Ejecutar las consultas SQL mediante un objeto `PreparedStatement`.
- Tratar los resultados obtenidos mediante un objeto `ResultSet`.
- Cerrar la conexión con la base de datos y otros objetos utilizados.

## Conexión

Para conectarnos a una base de datos MySQL necesitamos los siguientes datos:

- `url`: URL de la base de datos.
- `user`: Usuario de la base de datos.
- `password`: Contraseña del usuario de la base de datos.

In [ ]:
import java.sql.*;

Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");

Debemos capturar las posibles excepciones que puedan producirse al intentar establecer la conexión.

In [ ]:
import java.sql.*;

try {
    Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");
    conexion.close(); //Al finalizar, cerramos la conexión
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

Tras realizar nuestras consultas a la base de datos, debemos cerrar la conexión, para ello utilizamos el método `close()`.

Aunque podríamos evitarlo si utilizamos la construcción de Java `try-with-resources` que cierra automáticamente los recursos abiertos al finalizar el bloque `try`. En esta construcción, todos los recursos que se abran dentro del paréntesis de apertura del `try` se cerrarán automáticamente al finalizar el bloque `try`.

In [ ]:
try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password")) {
    //Aquí dentro realizamos las operaciones con la base de datos
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

# Operaciones CRUD

Las operaciones CRUD son las operaciones básicas que se pueden realizar sobre una base de datos: Create, Read, Update y Delete.

Para ello, utilizaremos la clase `PreparedStatement`. También existe la clase `Statement`, pero es más seguro utilizar `PreparedStatement` ya que evita la inyección SQL, por lo que solo utilizaremos esta última.

Un objeto de esta clase se genera de la siguiente manera:

```java
PreparedStatement ps = conexion.prepareStatement(sentenciaSQL);
```

Esta clase dispone de dos métodos para ejecutar consultas SQL: 

- `executeQuery()`. Se utiliza para consultas de tipo `SELECT` y devuelve un objeto `ResultSet` con los resultados obtenidos.
- `executeUpdate()`. Se utiliza para consultas de tipo `INSERT`, `UPDATE` y `DELETE` y devuelve un entero con el número de filas afectadas.

In [ ]:
String sql = "SELECT * FROM Alumnos";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");
     PreparedStatement sentencia = conexion.prepareStatement(sql);
     ResultSet resultado = sentencia.executeQuery()) {
    while (resultado.next()) {
        System.out.println(resultado.getString("nombre"));
    }
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

## Manejo del `ResultSet`

## Consultas preparadas

## Transacciones